# Import Packages & Data

In [1]:
# Importing required packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_rows = 1000

# Import Data

train = pd.read_csv('/Users/nishchay/Python/4. Ames Housing Dataset - Kaggle/home-data-for-ml-course/train.csv', index_col = 0)
test= pd.read_csv('/Users/nishchay/Python/4. Ames Housing Dataset - Kaggle/home-data-for-ml-course/test.csv', index_col = 0)
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
train['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

# Data Cleaning

## Checking Missing Values

In [3]:
columns_nan=pd.isnull(train).sum()
columns_nan = columns_nan[columns_nan>0].sort_values(ascending=False)
print(f'Number of Rows: {train.shape[0]}')
print(f'Number of columns: {train.shape[1]}')
print(f'Number of columns with missing values: {columns_nan.count()}')
print(f'Columns with number of missing values: \n{columns_nan}')

Number of Rows: 1460
Number of columns: 80
Number of columns with missing values: 19
Columns with number of missing values: 
PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64


# Create a List of features with - missing values, numerical values & categorical values

In [4]:
features= train.columns.tolist() # List of all features.
features_nan= columns_nan.index.tolist() # Features of features with missing values.
features_no_nan= [x for x in features if x not in features_nan] # List of features with no missing value.

num_features= [] # List of numerical features.
for x in features_no_nan:
    if train[x].dtype != object:
        num_features.append(x)  
        
cat_features=[] # List of categorical features.
for x in features_no_nan:
    if train[x].dtype==object:
        cat_features.append(x)

drop_features = [] # List of dropped features.

## Fill/ Drop missing values

In [5]:
#Electrical
train[pd.isnull(train.Electrical)]
train = train.drop([1380],axis=0) # Dropped one row with Nan in Electrical column.

pd.isnull(train.Electrical).sum() # Now electrical has no miising value.
cat_features.append('Electrical')
features_nan.remove('Electrical')

# MasVnrArea & MasVnrType

train['MasVnrArea'].fillna(0)

sns.set(font_scale=1)
fig, ax=plt.subplots(2,1,figsize=(10,10))
plt.figure(figsize=(14,6))
sns.regplot(data= train, x=train['MasVnrArea'], y= train['SalePrice'],ax=ax[0])
sns.boxplot(data= train, x=train['MasVnrType'], y= train['SalePrice'],ax=ax[1])

train[['MasVnrArea','SalePrice']].corr()

# MasVnrArea has low corelation to SalePrice. 
# And from the boxplot below MasVnrType has no direct affect on SalePrice.
# Drop both features.

drop_features.append(['MasVnrArea','MasVnrType'])
features_nan.remove('MasVnrArea')
features_nan.remove('MasVnrType')


In [6]:
# PoolQC, Fence, Alley, MiscFeature, GarageType, GarageFinish, GarageQual, GarageCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2 & FireplaceQu

# From data_description: nan refers to No pool/ Fence/ Alley/ MiscFeature.

a=['PoolQC','Fence','Alley', 'MiscFeature','GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu']

for x in a:
    train[a]=train[a].fillna('No')

for x in a:
    cat_features.append(x)

for x in a:
    features_nan.remove(x)

# For remaining Nan features, will use imputation later.
print(f'Remaining Nan Features: {features_nan}')
    

Remaining Nan Features: ['LotFrontage', 'GarageYrBlt', 'MasVnrArea', 'MasVnrType']


# Exploratory Data Analysis

## -- Numerical Features

In [7]:
len(num_features)

34

# Corelation Heatmap

cor=train[num_features].corr(method='pearson')

plt.figure(figsize=(200,200))
sns.set(font_scale=6)
sns.heatmap(data=cor[num_features], annot=True)

# Corelation between features-

1. OverallQual, GrLivArea have strong corelation with SalePrice.
2. GarageCar & GarageArea have strong corelation. (Drop GarageArea)
3. TotalBsmtSF & 1stFrSf have strong Corelation. (use 1 drop 1) 
4. YearBuilt & GarageYrBuilt have strong Corelation. (Drop GarageYrBuilt)
5. TotRmsAbvGrd & GrLivArea have strong Corelation. (Drop TotRmsAbvGrd)
6. YearBuilt & YearRemodAdd , TotalBsmtSF & 1stFrSf, FullBath, TotRmsAbvGrd, GarageCar & GarageArea have moderate corelation with SalePrice.

a = 20  # number of rows
b = 2  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(14,80))
sns.set(font_scale=1)

for i in num_features:
    plt.subplot(a, b, c)
    sns.regplot(data = train, x=train[i],y=train['SalePrice'])
    c = c + 1

plt.show()

### From the above scatterplots: '2ndFlrSF' & 'PoolArea' also has a linear relation to SalePrice.

## -- Categorical Features

In [8]:
print(cat_features)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition', 'Electrical', 'PoolQC', 'Fence', 'Alley', 'MiscFeature', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu']


In [9]:
# Check categories in each column.
for x in cat_features:
    print(f'{x}: {train[x].nunique()} : {train[x].unique()}')


MSZoning: 5 : ['RL' 'RM' 'C (all)' 'FV' 'RH']
Street: 2 : ['Pave' 'Grvl']
LotShape: 4 : ['Reg' 'IR1' 'IR2' 'IR3']
LandContour: 4 : ['Lvl' 'Bnk' 'Low' 'HLS']
Utilities: 2 : ['AllPub' 'NoSeWa']
LotConfig: 5 : ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
LandSlope: 3 : ['Gtl' 'Mod' 'Sev']
Neighborhood: 25 : ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Condition1: 9 : ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Condition2: 8 : ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
BldgType: 5 : ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
HouseStyle: 8 : ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
RoofStyle: 6 : ['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
RoofMatl: 8 : ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'

In [10]:
cat_eda = train[cat_features].copy()  
cat_eda['SalePrice'] = train['SalePrice']

for x in cat_features:
    a = cat_eda[[x, 'SalePrice']]
    a = a.groupby([x]).mean('SalePrice')
    a = a.sort_values("SalePrice")
    cat_eda.astype(str)

    o = 1
    for z in a.index:
        cat_eda[x] = cat_eda[x].replace(z, o)
        o = o + 1

cat_eda.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,GarageFinish,GarageQual,GarageCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,FireplaceQu,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,4,2,1,2,2,1,1,17,5,5,...,3,4,6,4,4,1,7,6,2,208500
2,4,2,1,2,2,2,1,21,3,5,...,3,4,6,4,4,4,5,6,4,181500
3,4,2,2,2,2,1,1,17,5,5,...,3,4,6,4,4,2,7,6,4,223500
4,4,2,2,2,2,3,1,18,5,5,...,2,4,6,3,5,1,5,6,5,140000
5,4,2,2,2,2,2,1,25,5,5,...,3,4,6,4,4,3,7,6,4,250000


cat_eda.corr(method='spearman')

plt.figure(figsize=(200,200))
sns.set(font_scale=6)
sns.heatmap(data=cat_eda.corr(), annot=True)

From the above heatmap:
1. Neighborhood, ExterQual, KitchenQual & BsmtQual has strong corelation to SalePrice.
2. GarageFinish, Foundation & FireplaceQu has moderate corelation to SalePrice.


a = 22  # number of rows
b = 2  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(14,80))
sns.set(font_scale=1)

for i in cat_features:
    plt.subplot(a, b, c)
    sns.regplot(data = cat_eda, x=cat_eda[i],y=cat_eda['SalePrice'])
    c = c + 1

plt.show()

In [11]:
use_cat= ['BldgType', 'CentralAir','Neighborhood', 'ExterQual', 'KitchenQual', 'BsmtQual', 'GarageFinish', 'GarageCond', 'Foundation', 'FireplaceQu']

ord_cat= ['FireplaceQu', "BsmtFinType1", 'CentralAir','Functional', 'ExterQual', 'KitchenQual', 'BsmtQual', 'GarageFinish', 'GarageCond', 'Foundation']
oh_cat= ['BldgType', 'Neighborhood']

use_features = ['BsmtUnfSF', 'OverallQual', 'GrLivArea', 'YearBuilt', 'GarageCars', 'TotalBsmtSF', "YearRemodAdd"]

use_features = use_features + use_cat
use_features.sort()
print(use_features)

['BldgType', 'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'ExterQual', 'FireplaceQu', 'Foundation', 'GarageCars', 'GarageCond', 'GarageFinish', 'GrLivArea', 'KitchenQual', 'Neighborhood', 'OverallQual', 'TotalBsmtSF', 'YearBuilt', 'YearRemodAdd']


# Model

## -- Encoding categorical features

In [12]:
import sklearn
from sklearn.model_selection import train_test_split
X = train[use_features].copy()
y = train.SalePrice

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)


# Ordinal encoding

from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

#### Make copy to avoid changing original data 
OH_train_X = train_X.copy()
OH_val_X = val_X.copy()

OH_train_X[use_cat] = ordinal_encoder.fit_transform(train_X[use_cat])

OH_val_X[use_cat] = ordinal_encoder.transform(val_X[use_cat])

print('Ordinal encoding \n')


# XGBOOST

from xgboost import XGBRegressor

modelx = XGBRegressor(max_depth = 4, n_estimators=65, learning_rate=0.1)

modelx.fit(OH_train_X, train_y)

predictions_x = modelx.predict(OH_val_X)

print(f'XGB Score: {modelx.score(OH_train_X,train_y)}')

from sklearn.metrics import mean_absolute_error

print(f'XGB MAE: {mean_absolute_error(val_y, predictions_x)}\n')

from sklearn.metrics import mean_squared_log_error
print(f'RMSLE: {np.sqrt(mean_squared_log_error(val_y, predictions_x))}\n')



Ordinal encoding 

XGB Score: 0.9612676907404515
XGB MAE: 15337.089704623288

RMSLE: 0.13796467019295466



# Run Model on whole train set

In [13]:
X = train[use_features].copy()
y = train.SalePrice

# Ordinal encoding

from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()


X[use_cat] = ordinal_encoder.fit_transform(X[use_cat])


# XGBOOST

from xgboost import XGBRegressor

modelx = XGBRegressor(max_depth = 4, n_estimators=65, learning_rate=0.1)

modelx.fit(X,y)



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=65, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
# Making Predictions

In [15]:
wanted_test_data = test[use_features]

In [16]:
wanted_test_data.isnull().sum()

BldgType          0
BsmtQual         44
BsmtUnfSF         1
CentralAir        0
ExterQual         0
FireplaceQu     730
Foundation        0
GarageCars        1
GarageCond       78
GarageFinish     78
GrLivArea         0
KitchenQual       1
Neighborhood      0
OverallQual       0
TotalBsmtSF       1
YearBuilt         0
YearRemodAdd      0
dtype: int64

In [17]:
### Missing value imputation in test data.

In [18]:
na_cols = ['GarageCond', 'GarageFinish', 'BsmtQual', 'FireplaceQu']
wanted_test_data[na_cols] = wanted_test_data[na_cols].fillna('No')

wanted_test_data['TotalBsmtSF'] = wanted_test_data['TotalBsmtSF'].fillna(0)

wanted_test_data['KitchenQual'] = wanted_test_data['KitchenQual'].fillna('TA')

mean = wanted_test_data['GarageCars'].mean()
wanted_test_data['GarageCars'] = wanted_test_data['GarageCars'].fillna(mean)

mode = wanted_test_data['BsmtUnfSF'].mean()
wanted_test_data['BsmtUnfSF'] = wanted_test_data['BsmtUnfSF'].fillna(mode)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-18-4ade0b56bba0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wanted_test_data['TotalBsmtSF'] = wanted_test_data['TotalBsmtSF'].fillna(0)
<ipython-input-18-4ade0b56bba0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [19]:
wanted_test_data.isnull().sum()

BldgType        0
BsmtQual        0
BsmtUnfSF       0
CentralAir      0
ExterQual       0
FireplaceQu     0
Foundation      0
GarageCars      0
GarageCond      0
GarageFinish    0
GrLivArea       0
KitchenQual     0
Neighborhood    0
OverallQual     0
TotalBsmtSF     0
YearBuilt       0
YearRemodAdd    0
dtype: int64

In [20]:
wanted_test_data[use_cat] = ordinal_encoder.fit_transform(wanted_test_data[use_cat])

<ipython-input-20-54372d3edeba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wanted_test_data[use_cat] = ordinal_encoder.fit_transform(wanted_test_data[use_cat])
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [21]:
prediction= modelx.predict(wanted_test_data)

In [22]:
sample_submission = pd.read_csv('/Users/Nishchay/Python/4. Ames Housing Dataset - Kaggle/home-data-for-ml-course/sample_submission.csv')
submission = sample_submission
submission.SalePrice = prediction
#submission = submission.set_index('Id')
submission.head()
submission.to_csv('/Users/Nishchay/Python/4. Ames Housing Dataset - Kaggle/home-data-for-ml-course/submission 2.csv', index = False)